In [11]:
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env.

True

In [13]:
from langchain.chat_models import ChatOpenAI
KEY = os.getenv("apikey")


In [16]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2



In [23]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [24]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use lit as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json} 
"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [27]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

e:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\mcqenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [28]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
    if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [30]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=[ "subject",  "quiz"],
    template=TEMPLATE2
    )

In [31]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [32]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [33]:
file_path=r"E:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\data.txt"

In [36]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [38]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]

From a theoreti

In [40]:
# Serialize the Python dictionary into a JSON-formatted string
print(json.dumps(RESPONSE_JSON))

{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


In [41]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"


In [42]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

e:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\mcqenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) fiel

In [43]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1266
Prompt Tokens:888
Completion Tokens:378
Total Cost:0.002088


In [44]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]\n\

In [45]:
quiz=response.get("quiz")

In [46]:
quiz=json.loads(quiz)
quiz

{'1': {'mcq': 'What is the main goal of machine learning?',
  'options': {'a': 'To follow explicit instructions',
   'b': 'To learn from data and generalize to unseen data',
   'c': 'To ignore data and rely on pre-defined rules',
   'd': 'To perform tasks without any data'},
  'correct': 'b'},
 '2': {'mcq': 'Which field of study focuses on exploratory data analysis through unsupervised learning?',
  'options': {'a': 'Machine Learning',
   'b': 'Artificial Intelligence',
   'c': 'Data Mining',
   'd': 'Predictive Analytics'},
  'correct': 'c'},
 '3': {'mcq': 'What is the mathematical foundation of machine learning methods?',
  'options': {'a': 'Data Mining',
   'b': 'Mathematical Optimization',
   'c': 'Computational Statistics',
   'd': 'Artificial Neural Networks'},
  'correct': 'b'},
 '4': {'mcq': 'In which field does machine learning find application in speech recognition?',
  'options': {'a': 'Agriculture',
   'b': 'Natural Language Processing',
   'c': 'Computer Vision',
   'd': '

In [47]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [48]:
quiz_table_data

[{'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a: To follow explicit instructions | b: To learn from data and generalize to unseen data | c: To ignore data and rely on pre-defined rules | d: To perform tasks without any data',
  'Correct': 'b'},
 {'MCQ': 'Which field of study focuses on exploratory data analysis through unsupervised learning?',
  'Choices': 'a: Machine Learning | b: Artificial Intelligence | c: Data Mining | d: Predictive Analytics',
  'Correct': 'c'},
 {'MCQ': 'What is the mathematical foundation of machine learning methods?',
  'Choices': 'a: Data Mining | b: Mathematical Optimization | c: Computational Statistics | d: Artificial Neural Networks',
  'Correct': 'b'},
 {'MCQ': 'In which field does machine learning find application in speech recognition?',
  'Choices': 'a: Agriculture | b: Natural Language Processing | c: Computer Vision | d: Speech Recognition',
  'Correct': 'd'},
 {'MCQ': 'Which framework provides a theoretical viewpoint for desc

In [49]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is the main goal of machine learning?,a: To follow explicit instructions | b: To lea...,b
1,Which field of study focuses on exploratory da...,a: Machine Learning | b: Artificial Intelligen...,c
2,What is the mathematical foundation of machine...,a: Data Mining | b: Mathematical Optimization ...,b
3,In which field does machine learning find appl...,a: Agriculture | b: Natural Language Processin...,d
4,Which framework provides a theoretical viewpoi...,a: Computational Statistics | b: Probably Appr...,b


In [50]:
quiz.to_csv("machinelearning.csv",index=False)
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_03_2024_12_22_31'